### Road Construction

Road construction is narrowing University Avenue to a single lane: vehicles can drive on University Avenue south to north or north to south. The lane is long enough that arbitrarily many vehicles can drive on it. If vehicles drive in both directions, they collide. The task is to synchronize vehicles such that once a vehicle enters University Avenue, it can continue to its end without colliding with a vehicle driving in the opposite direction.

The lane has similarities to a critical section. Any solution to the critical section problem has to satisfy three safety properties and one liveness property:

1. Mutual exclusion
2. No deadlock (no livelock)
3. No unnecessary delay
4. Eventual entry

What are these four properties in terms of the Road Construction problem? [2 points]

1. no 2 opposite direction cars will be in the critical section at the same time
2. no car waiting to go south is waiting for a car to go north, and no car waiting to go south is waiting for a car to go north. ie. they are both waiting for eachother but none of them go
3. no car waits at the avenue if none are in the critical section. ie. if critical section is empty a car will immediately enter
4. all cars will eventually enter the critical section. ie. no car will be left starved

Implement the Road Construction problem in Python with semaphores by completing the template below. Your solution must satisfy properties 1 - 4. You must not modify the existing code and must not import libraries (there is no need). You can add further code, and you can add further cells for testing as you like. Only the code in the cell below will be graded. Some `write` statements are included to illustrate the progress. For example, the output may start with:

```
9↑
 ↑9
3↓
7↓
 ↓7
 ↓3
5↑
0↑
2↑
8↑
```

- Add comments explaining your design, in particular how eventual entry is ensured. An uncommented program will not get full marks.
- The implementation represents the two directions by `0` and `1`. If `d` is the direction of the vehicle, the opposite direction is `1 - d`.
- Variables `go`, `driving`, and `waiting` are indexed by the direction. Use these variables.

Answer this together with the next question. [5 points]

In [16]:
from threading import Thread, Semaphore
from time import sleep
from random import random, randint
from sys import stdout

# eventual entry is ensured due to the use of semaphores
# a car will wait until it has acquired the go/mutex semaphore which will it then use
# to enter the critical section asuming it has met all its other conditions. ie. no car in opposing direction



def arrive(direction):
    if driving[1-direction] > 0: waiting[direction] += 1; go[direction].acquire()
    mutex.acquire()
    if waiting[direction] > 0: waiting[direction] -=1
    driving[direction] += 1
    mutex.release()  
    
def leave(direction):
    mutex.acquire()
    driving[direction] -= 1
    if driving[direction] == 0: go[1-direction].release()
    mutex.release()

class Vehicle(Thread):
    def __init__(self, name, direction):
        self.n, self.d = name, direction; Thread.__init__(self)
    def run(self):
        sleep(random()) # wait up to 1 sec before trying to enter lane
        arrive(self.d)
        stdout.write(self.n + ("↓\n" if self.d else "↑\n")) # output after the lane is entered
        sleep(random()) # drive through lane for up to 1 sec
        stdout.write((" ↓" if self.d else " ↑") + self.n + "\n") # output before leaving the lane
        leave(self.d)

go = [Semaphore(0), Semaphore(0)] # semaphores for entering the lane
driving = [0, 0] # number of vehicles driving in each direction
waiting = [0, 0] # number of vehicles waiting in each direction
mutex = Semaphore(1)

for v in range(10): Vehicle(str(v), randint(0, 1)).start()

State the following properties in terms of the variables `go`, `driving`, and `waiting`. Your implementation must ensure that these are global invariants.

1. All elements of `driving` and `waiting` must not be negative.
2. Vehicles can't be driving in both directions
3. Semaphore `go[direction]` is positive if and only if there are no vehicles driving in the other direction, but there are vehicles waiting in the same direction.

You may use mathematical notation or Python notation, as long as you are consistent (and it is clear what equality is). Symbols to copy and paste: ⟨ ⟩ ¬ ∧ ∨ ≠ ≡ ⇒ ⇐ ≤ ≥ [2 points]

1. driving[direction] > 0 ∧ waiting[direction] > 0
2. driving[direction] > 0 ⇒ driving[1-direction] = 0
3. go[direction] > 0 ⇒ driving[1-direction] = 0  ∧ waiting[direction] > 0

Can vehicles driving in the same direction overtake each other? Answer Yes or No and give a brief explanation. [1 point]

yes. there is no mechanism in place to ensure that the vehicle which enters the road first finishes first since it sleeps for a random time, a vehicle entering after can have a smaller sleep and finish first.